# From https://github.com/theislab/scTab/blob/devel/notebooks/store_creation/05_compute_pca.ipynb

In [8]:
import os
from os.path import join

import dask.dataframe as dd
import dask.array as da
import numpy as np
import pandas as pd
import pickle
from dask_ml.decomposition import IncrementalPCA
import scanpy as sc
import torch
from math import ceil

### CellNet

In [3]:
PATH = '/lustre/groups/ml01/workspace/till.richter/merlin_cxg_2023_05_15_sf-log1p'

In [4]:
def get_count_matrix(ddf):
    x = (
        ddf['X']
        .map_partitions(
            lambda xx: pd.DataFrame(np.vstack(xx.tolist())), 
            meta={col: 'f4' for col in range(19331)}
        )
        .to_dask_array(lengths=[1024] * ddf.npartitions)
    )

    return x

Compute PCA for consistency with 64 dimensions in trained models

In [7]:
os.makedirs(join(PATH, 'pca'), exist_ok=True)


n_comps = 64


for split in ['test']:  # we only need test
    if os.path.exists(join(PATH, 'pca', f'x_pca_{split}_{n_comps}.npy')):
        print('Split ', split, 'already exists')
        continue
    x = get_count_matrix(dd.read_parquet(join(PATH, split), split_row_groups=True))
    pca = IncrementalPCA(n_components=n_comps, iterated_power=3)
    x_pca = da.compute(pca.fit_transform(x))[0]
    with open(join(PATH, 'pca', f'x_pca_{split}_{n_comps}.npy'), 'wb') as f:
        np.save(f, x_pca)

Split  test already exists


In [13]:
root = os.path.dirname(os.path.dirname(os.path.abspath(os.path.curdir)))
hvg_indices = pickle.load(open(root + '/self_supervision/data/hvg_2000_indices.pickle', 'rb'))

In [14]:
os.makedirs(join(PATH, 'pca'), exist_ok=True)


n_comps = 64


for split in ['test']:
    if os.path.exists(join(PATH, 'pca', f'x_hvg_pca_{split}_{n_comps}.npy')):
        print('Split ', split, 'already exists')
        continue
    x = get_count_matrix(dd.read_parquet(join(PATH, split), split_row_groups=True))
    x = x[:, hvg_indices]
    pca = IncrementalPCA(n_components=n_comps, iterated_power=3)
    x_pca = da.compute(pca.fit_transform(x))[0]
    with open(join(PATH, 'pca', f'x_hvg_pca_{split}_{n_comps}.npy'), 'wb') as f:
        np.save(f, x_pca)

In [5]:
# For subsampled train data

def get_sampled_files(base_path, split, sub_sample_frac):
    files = [file for file in os.listdir(os.path.join(base_path, split)) if file.endswith('.parquet')]
    files = [os.path.join(base_path, split, file) for file in sorted(files, key=lambda x: int(x.split('.')[1]))]
    num_files_to_sample = ceil(sub_sample_frac * len(files))
    return files[:num_files_to_sample]

In [6]:
def get_count_matrix(files):
    x = (
        dd.read_parquet(files, engine='pyarrow')
        .map_partitions(
            lambda df: pd.DataFrame(np.vstack(df['X'].tolist())), 
            meta={col: 'f4' for col in range(19331)}
        )
        .to_dask_array(lengths=True)
    )
    return x

In [9]:
n_comps = 64
sub_sample_frac = 0.2  # 20% subsample
train_files = get_sampled_files(PATH, 'train', sub_sample_frac)

# Create a directory for PCA results
os.makedirs(os.path.join(PATH, 'pca'), exist_ok=True)

# Check if PCA has already been computed for the subsampled train data
pca_path = os.path.join(PATH, 'pca', f'x_pca_20p_subsample_train_{n_comps}.npy')
if os.path.exists(pca_path):
    print('PCA for subsampled train data already exists')
else:
    x = get_count_matrix(train_files)
    pca = IncrementalPCA(n_components=n_comps, iterated_power=3)
    # Compute PCA in an incremental way to handle large data
    x_pca = da.compute(pca.fit_transform(x))[0]
    with open(pca_path, 'wb') as f:
        np.save(f, x_pca)
    print('PCA for subsampled train data computed and saved.')

PCA for subsampled train data computed and saved.


### OOD - Tail of Hippocampus

In [12]:
DATA_PATH = '/lustre/groups/ml01/workspace/till.richter/'
adata = sc.read_h5ad(os.path.join(DATA_PATH, 'tail_of_hippocampus', 'tail_of_hippocampus.h5ad'))

In [13]:
# Load adata
print('Loaded raw adata with shape: ', adata.X.shape)

# Load CellNet genes
cellnet_genes_path = os.path.join(PATH, 'var.parquet')
cellnet_genes = list(pd.read_parquet(cellnet_genes_path)['feature_id'])
print("Loaded CellNet genes.")

# Find common and missing genes
common_genes = list(set(adata.var['Gene'].index) & set(cellnet_genes))
missing_genes = list(set(cellnet_genes) - set(adata.var['Gene'].index))
print(f"Found {len(common_genes)} common genes and {len(missing_genes)} missing genes.")

# Create a dictionary to map 'Gene' to 'ensembl_ids'
gene_to_ensembl = dict(zip(adata.var['Gene'].index, adata.var_names))

# Convert common genes to their corresponding ensembl IDs
common_ensembl_ids = [gene_to_ensembl[gene] for gene in common_genes]

# Filter and reorder genes
adata = adata[:, common_ensembl_ids]
print("Filtered and reordered genes. New adata shape: ", adata.X.shape)

# Normalize and log transform
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
print("Normalized and log-transformed data.")

# Load cell_type_mapping
cell_type_mapping = pd.read_parquet(os.path.join(PATH, 'categorical_lookup/cell_type.parquet'))

# Create mapping dictionary for cell_type to int64 encoding
cell_type_to_encoding = {cell_type: idx for idx, cell_type in cell_type_mapping['label'].items()}

# Filter cells by valid cell types
valid_cell_types = set(cell_type_to_encoding.keys())
adata = adata[adata.obs['cell_type'].isin(valid_cell_types)]
print("Filtered cells by valid cell types. New adata shape: ", adata.X.shape)

# Encode cell types
y_adata = np.array([cell_type_to_encoding[cell_type] for cell_type in adata.obs['cell_type'].values])

# Zero-padding
if missing_genes:
    zero_padding_df = pd.DataFrame(
        data=0,
        index=adata.obs.index,
        columns=missing_genes
    )

    concatenated_df = pd.concat([adata.to_df(), zero_padding_df], axis=1)
    concatenated_df = concatenated_df[cellnet_genes]  # Ensure ordering of genes

    # Create new AnnData object to ensure consistency
    adata = sc.AnnData(X=concatenated_df.values, 
                        obs=adata.obs,
                        var=pd.DataFrame(index=cellnet_genes))

# Double-check that the genes are in the correct order
assert all(adata.var_names == cellnet_genes), 'Genes are not in the correct order.'

print('Final shape of adata: ', adata.X.shape)


# PyTorch DataLoader
# Assuming you have a function called `cell_type_to_encoding` to convert cell_type to int64
tensor_x = torch.Tensor(adata.X)
tensor_y = torch.Tensor(adata.obs['cell_type'].map(cell_type_to_encoding).values).type(torch.int64)

Loaded raw adata with shape:  (56367, 59357)
Loaded CellNet genes.
Found 18956 common genes and 375 missing genes.
Filtered and reordered genes. New adata shape:  (56367, 18956)
Normalized and log-transformed data.
Filtered cells by valid cell types. New adata shape:  (56281, 18956)
Final shape of adata:  (56281, 19331)


In [14]:
tensor_x.shape

torch.Size([56281, 19331])

In [15]:
from sklearn.decomposition import PCA
pca = PCA(n_components=64)
x_pca = pca.fit_transform(tensor_x)
with open(join(DATA_PATH, 'tail_of_hippocampus', 'pca', f'x_tail_of_hippocampus_pca_64.npy'), 'wb') as f:
        np.save(f, x_pca)

### OOD - Non neuronal cells

In [3]:
DATA_PATH = '/lustre/groups/ml01/workspace/till.richter/'
adata = sc.read_h5ad(os.path.join(DATA_PATH, 'non_neuronal', 'non_neuronal.h5ad'))

In [4]:
# Load adata
print('Loaded raw adata with shape: ', adata.X.shape)

# Load CellNet genes
cellnet_genes_path = os.path.join(PATH, 'var.parquet')
cellnet_genes = list(pd.read_parquet(cellnet_genes_path)['feature_id'])
print("Loaded CellNet genes.")

# Find common and missing genes
common_genes = list(set(adata.var['Gene'].index) & set(cellnet_genes))
missing_genes = list(set(cellnet_genes) - set(adata.var['Gene'].index))
print(f"Found {len(common_genes)} common genes and {len(missing_genes)} missing genes.")

# Create a dictionary to map 'Gene' to 'ensembl_ids'
gene_to_ensembl = dict(zip(adata.var['Gene'].index, adata.var_names))

# Convert common genes to their corresponding ensembl IDs
common_ensembl_ids = [gene_to_ensembl[gene] for gene in common_genes]

# Filter and reorder genes
adata = adata[:, common_ensembl_ids]
print("Filtered and reordered genes. New adata shape: ", adata.X.shape)

# Normalize and log transform
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
print("Normalized and log-transformed data.")

# Load cell_type_mapping
cell_type_mapping = pd.read_parquet(os.path.join(PATH, 'categorical_lookup/cell_type.parquet'))

# Create mapping dictionary for cell_type to int64 encoding
cell_type_to_encoding = {cell_type: idx for idx, cell_type in cell_type_mapping['label'].items()}

# Filter cells by valid cell types
valid_cell_types = set(cell_type_to_encoding.keys())
adata = adata[adata.obs['cell_type'].isin(valid_cell_types)]
print("Filtered cells by valid cell types. New adata shape: ", adata.X.shape)

# Encode cell types
y_adata = np.array([cell_type_to_encoding[cell_type] for cell_type in adata.obs['cell_type'].values])

# Zero-padding
if missing_genes:
    zero_padding_df = pd.DataFrame(
        data=0,
        index=adata.obs.index,
        columns=missing_genes
    )

    concatenated_df = pd.concat([adata.to_df(), zero_padding_df], axis=1)
    concatenated_df = concatenated_df[cellnet_genes]  # Ensure ordering of genes

    # Create new AnnData object to ensure consistency
    adata = sc.AnnData(X=concatenated_df.values, 
                        obs=adata.obs,
                        var=pd.DataFrame(index=cellnet_genes))

# Double-check that the genes are in the correct order
assert all(adata.var_names == cellnet_genes), 'Genes are not in the correct order.'

print('Final shape of adata: ', adata.X.shape)


# PyTorch DataLoader
# Assuming you have a function called `cell_type_to_encoding` to convert cell_type to int64
tensor_x = torch.Tensor(adata.X)
tensor_y = torch.Tensor(adata.obs['cell_type'].map(cell_type_to_encoding).values).type(torch.int64)

Loaded raw adata with shape:  (888263, 59357)
Loaded CellNet genes.
Found 18956 common genes and 375 missing genes.
Filtered and reordered genes. New adata shape:  (888263, 18956)
Normalized and log-transformed data.
Filtered cells by valid cell types. New adata shape:  (871418, 18956)
Final shape of adata:  (871418, 19331)


In [5]:
tensor_x.shape

torch.Size([871418, 19331])

In [6]:
from sklearn.decomposition import PCA
pca = PCA(n_components=64)
x_pca = pca.fit_transform(tensor_x)
with open(join(DATA_PATH, 'non_neuronal', 'pca', 'x_non_neuronal_pca_64.npy'), 'wb') as f:
        np.save(f, x_pca)

In [ ]:
x_dask = da.from_array(tensor_x, chunks=(1000, x.shape[1]))
x_dask = x_dask.astype('float64')

# Initialize Incremental PCA
ipca = IncrementalPCA(n_components=64)

# Fit and Transform
x_pca_dask = ipca.fit_transform(x_dask)

# Save to disk
with open(join(DATA_PATH, 'non_neuronal', 'pca', 'x_non_neuronal_pca_64.npy'), 'wb') as f:
    np.save(f, x_pca_dask)